# Farina in Natura - estratégia

![](Logo_farina_in_natura_edited.jpg)

## Importando os dados e observações iniciais

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import nbformat
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode (connected = True )
from IPython.display import display


In [2]:
from fpdf import FPDF

In [3]:
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
# para resolver o problema: "(unicode error) 'utf-8' codec can't decode "
# https://www.youtube.com/watch?v=ZSC7X4N8RKo,
#Erro: 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 900: invalid continuation byte
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="utf-8", sep = ';')
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="ISO-8859-1", sep = ';')

transactions = pd.read_csv(r'C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\CSV\farina.CSV', 
encoding="ISO-8859-1", sep = ';')

# Limpeza e manipulação dos dados

In [5]:
### CHECK OUT IF THE NUMBER OF TOTAL ENTRIES IS EQUAL TO THE NUMBER OF ENTRIES OF THE VARIABLES  

transactions.isnull().sum()

venda            6
data_hora        6
cliente          6
mesa          3219
item            76
produto          6
quantidade       6
valor_unit       6
desconto         6
total_item       6
dtype: int64

In [6]:
## DROPPING THE ROWS THAT ALL COLUMN VALUES ARE NULL
#  https://www.youtube.com/watch?v=uDr67HBIPz8&t=818s

transactions = transactions.dropna(how='all')

In [7]:
# Delete the column "item"

del transactions["item"]

In [8]:
## Using isna() to select all rows with NaN in the column ('mesa') and the value "CONSUMIDOR" on the column "cliente" 
## and replace the "mesa" value with "M0"
## https://datatofish.com/rows-with-nan-pandas-dataframe/
## https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/

transactions.loc[(transactions["cliente"] == 'CONSUMIDOR') &  (transactions['mesa'].isna()), 'mesa']= 'M0'


In [9]:
# Checking if the changes were correct.

transactions.loc[transactions["mesa"] == 'M0']

,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item
121,520.0,02/02/2022 16:57,CONSUMIDOR,M0,BOMBOLONI NUTELLA 80G,2.0,12,0.0,24
122,520.0,02/02/2022 16:57,CONSUMIDOR,M0,TARTELETE DE MORANGO 120G,1.0,18,0.0,18
123,520.0,02/02/2022 16:57,CONSUMIDOR,M0,CROISSANT MUSSATRELA DE,1.0,"25,5",0.0,"25,5"
894,856.0,11/02/2022 19:19,CONSUMIDOR,M0,CROISSANT BRIE PARMA MEL DE,2.0,"25,5",0.0,51
895,856.0,11/02/2022 19:19,CONSUMIDOR,M0,VH SANTA HORTENSIA SAUVIG,1.0,59,0.0,59


In [10]:
# Assigning a value "ND" to the empty records in column "mesa"
# https://www.w3schools.com/python/pandas/pandas_cleaning_empty_cells.asp

transactions['mesa'].fillna("ND",inplace = True )

In [11]:
# Convert the "venda" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['venda'] = np.int64(transactions['venda'])

In [12]:
# Convert the "quantidade" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['quantidade'] = np.int64(transactions['quantidade'])

In [13]:
# change the decimal separator of the the variable "valor_unit" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['valor_unit'] = transactions['valor_unit'].str.replace(",",".")

# Convert the "valor_uni" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['valor_unit'] = transactions['valor_unit'].astype(float)

In [14]:
# change the decimal delimiter of the the variable "total_item" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['total_item'] = transactions['total_item'].str.replace(",",".")

# Convert the "total_item" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['total_item'] = transactions['total_item'].astype(float)

In [15]:


# Convert "data_hora" to datetime object
transactions['data_hora'] = pd.to_datetime(transactions['data_hora'], format='%d/%m/%Y %H:%M')

# Creating a horario column that is a copy of data_hora and coverting it to datetime and extracting only the day;
transactions['horario'] = (transactions['data_hora'])
transactions['horario'] = transactions['horario'].dt.date

#How to Solve Python ValueError: unconverted data remains
#read://https_researchdatapod.com/?url=https%3A%2F%2Fresearchdatapod.com%2Fhow-to-solve-python-valueerror-unconverted-data-remains%2F

In [16]:

# https://www.dataquest.io/blog/python-datetime-tutorial/


# add hora and minuto column; using class attributes .hour and .minute

transactions['hora'] = transactions.data_hora.dt.hour
transactions['minuto'] = transactions.data_hora.dt.minute



### Creating a column name 'hora_minuto'

In [17]:
# https://gist.github.com/krisbolton/aef6179ecf906892565d326b979b3a5b
# https://www.youtube.com/watch?v=TbXaP56Omuc


transactions['hora_minuto']=pd.to_datetime(transactions['data_hora'])

transactions['hora_minuto'] = transactions['hora_minuto'].dt.time

### Renaming the column "data_hora" to "data"

In [18]:
# https://www.statology.org/pandas-rename-columns/
# renaming the column

transactions.rename(columns = {'data_hora':'data'}, inplace = True)


### Extracting only the Date from 'data' column

In [19]:

## https://datagy.io/pandas-extract-date-from-datetime/
transactions['data'] = transactions['data'].dt.normalize()

In [20]:
# Definindo a função time_of_day
def time_of_day(hora):
    """
    Determina se a  compra foi feita pela manhã, tarde ou noite.
    """
    
    if hora < 12:
        
        return 'manhã'
    
    elif hora < 17:
        
        return 'tarde'
    
    else:
        
        return 'noite'

In [21]:
# Apply time of day function to 'hora' column

transactions['time_of_day'] = transactions.hora.apply(time_of_day)


In [22]:
# https://pt.stackoverflow.com/questions/508484/mostrar-dias-da-semana#:~:text=Como%20weekday()%20retorna%20um,fim%20de%20semana%20ou%20n%C3%A3o.

import locale
# setar locale para português para que os dias da semana retornem em portugês no código abaixo.
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [23]:
# Add a day of week variable and a second classifier of weekday or weekend

from datetime import date
import calendar

transactions['day_of_week'] = [calendar.day_name[i.weekday()] for i in transactions['horario']]


In [24]:

transactions['day_type'] = ['FDS' if (i == 'domingo') | (i == 'sábado') else 'DDS' for i in 
                                                                            transactions['day_of_week']]


In [25]:
transactions.rename(columns = {'day_of_week':'dia_da_semana'}, inplace = True)

In [26]:
transactions.rename(columns = {'time_of_day':'hora_do_dia'}, inplace = True)

In [27]:
transactions.rename(columns = {'day_type':'tipo_de_dia'}, inplace = True)

In [28]:
# Add an item count variable equal to one for future groupby operations

transactions['item_count'] = 1


# Verificando divergência dos dados

In [29]:
# Verificando se o valor "total_item" é igual a: "quantidade" * "valor_unit"

transactions["total_item_verif"] = transactions['quantidade']* transactions['valor_unit']

In [30]:
# Continuando o item anterior
# https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

transactions['confirm'] = np.where(transactions['total_item'] > transactions['total_item_verif'], True, False)


#### Creating a Column diference

In [31]:

transactions['diference']=(transactions['total_item'])-(transactions['total_item_verif'])

In [32]:
# continuando o item anterior
resumo = transactions.loc[transactions["confirm"] == True, ["total_item", "total_item_verif", 'diference']].sum()

print (resumo)

total_item          53878.3
total_item_verif    22219.1
diference           31659.2
dtype: float64


### Gerando um relatório em pdf dos dados divergentes

In [33]:

# How do I select specific rows and columns from a DataFrame?
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-select-specific-rows-and-columns-from-a-dataframe
# https://www.geeksforgeeks.org/select-rows-columns-by-name-or-index-in-pandas-dataframe-using-loc-iloc/

ts2 = transactions.loc[transactions["confirm"] == True, ["venda", "produto", "quantidade", "valor_unit", "total_item", "total_item_verif"]]

In [34]:
# Exporting to a pdf file the transactions that has a inconsistent data.
from matplotlib.backends.backend_pdf import PdfPages


In [35]:
# https://levelup.gitconnected.com/how-to-write-a-pandas-dataframe-as-a-pdf-5cdf7d525488

def _draw_as_table(df, pagesize):
    alternating_colors = [['white'] * len(df.columns), ['lightgray'] * len(df.columns)] * len(df)
    alternating_colors = alternating_colors[:len(df)]
    fig, ax = plt.subplots(figsize=pagesize)
    ax.axis("tight")
    ax.axis('off')
    the_table = ax.table(cellText=df.values,
                        rowLabels=df.index,
                        colLabels=df.columns,
                        rowColours=['lightblue']*len(df),
                        colColours=['lightblue']*len(df.columns),
                        cellColours=alternating_colors,
                        loc='center')
    return fig


In [36]:
# https://www.alixaprodev.com/2022/05/convert-pandas-dataframe-to-pdf-in-python.html
#fig, ax =plt.subplots(figsize=(14,6))
#ax.axis('equal')
#ax.axis('off')
#the_table = ax.table(cellText=ts2.values,colLabels=ts2.columns,loc='center')




In [37]:
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                fig.text(0.5, 0.5/pagesize[0],
                         "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                         ha='center', fontsize=8)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

In [38]:
### Trying to Align the page number
# https://www.geeksforgeeks.org/matplotlib-figure-figure-text-in-python/
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            #if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                #fig.text(0.5, 0.5/pagesize[0],
                       # "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                       # verticalalignment ='bottom', 
                       # horizontalalignment ='right',
                       #transform = ax.transAxes,
                        #color ='green',
                        #fontsize = 5
                         #)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

#### Creating a pdf file of the divergent data (ts2 dataframe)

In [39]:
dataframe_to_pdf(ts2, "dados_divergentes.pdf", numpages=(12, 1), pagesize=(11.7, 8.3))

#### Creating a HTML file of the divergent data ("ts2" dataframe).


In [40]:

f = open('dados_divergentes.html','w')
a = ts2.to_html()
f.write(a)
f.close()

In [41]:
# drop the 'confirm" and "total_item_verif" column
transactions = transactions.drop(['confirm'], axis=1)
transactions = transactions.drop(['total_item_verif'], axis=1)
transactions = transactions.drop (['diference'], axis=1)

> # **Exploração dos Dados**

In [42]:
# Find Unique Values of each item sold in the Column "produto"
# https://www.statology.org/pandas-unique-values-in-column/
# Pandas: get first 10 elements of a series
top_produtos = transactions.produto.value_counts()
print (top_produtos[:10])

CROISSANT                       2393
PASTEL DE NATA 60G              1339
CROISSANT  AMENDOAS 100G         742
FOLHADO MACA 80G                 719
SANDUICHE DE PEPERONI BRIE       596
COOKIES 80G                      559
CROISSANT IFOOD                  534
SOURDOUGH AZEITONA E ALECRIM     517
FOLHADO FRANGO 160G              500
BOMBOLONI DOCE DE LEITE 80G      487
Name: produto, dtype: int64


### Converting a Series to a dataframe and changing the namee of the column

In [43]:
top_produtos_df = top_produtos.to_frame()
top_produtos_df.rename(columns = {'produto':'quantidade'}, inplace = True)


In [44]:
f = open('top_produtos.html','w')
a = top_produtos_df.to_html()
f.write(a)
f.close()

In [45]:
# find the percentage each item is selling
top_produtos_perc = transactions.produto.value_counts(True)
print (top_produtos_perc[:10])

CROISSANT                       0.092067
PASTEL DE NATA 60G              0.051516
CROISSANT  AMENDOAS 100G        0.028547
FOLHADO MACA 80G                0.027662
SANDUICHE DE PEPERONI BRIE      0.022930
COOKIES 80G                     0.021507
CROISSANT IFOOD                 0.020545
SOURDOUGH AZEITONA E ALECRIM    0.019891
FOLHADO FRANGO 160G             0.019237
BOMBOLONI DOCE DE LEITE 80G     0.018737
Name: produto, dtype: float64


#### Converting to a Dataframe

In [46]:
top_produtos_perc_df = top_produtos_perc.to_frame()
top_produtos_perc_df.rename(columns = {'produto':'Proporção'}, inplace = True)

In [47]:
top_produtos_perc_df[:10]

,Proporção
CROISSANT,0.092067
PASTEL DE NATA 60G,0.051516
CROISSANT AMENDOAS 100G,0.028547
FOLHADO MACA 80G,0.027662
SANDUICHE DE PEPERONI BRIE,0.022930
COOKIES 80G,0.021507
CROISSANT IFOOD,0.020545
SOURDOUGH AZEITONA E ALECRIM,0.019891
FOLHADO FRANGO 160G,0.019237
BOMBOLONI DOCE DE LEITE 80G,0.018737


In [48]:
f = open('top_produtos_perc.html','w')
a = top_produtos_perc_df.to_html()
f.write(a)
f.close()

In [49]:
## Get unique names in the "produto" column using Series.unique() 
## https://sparkbyexamples.com/pandas/pandas-count-unique-values-in-column/
num_itens_vend = transactions.produto.unique().size

print('O número de diferentes itens vendido pela loja é:', num_itens_vend)

O número de diferentes itens vendido pela loja é: 212


In [50]:
# Look at the average number of items purchased...

avg_items = transactions.groupby(by='venda')['produto'].count().mean()

print('A média de itens por venda é: %.2f itens' %avg_items)

A média de itens por venda é: 2.79 itens


In [51]:
# ...and how that varies by time of day

single_transaction = pd.DataFrame(transactions.groupby(by='venda')['item_count'].sum())

single_transaction = single_transaction.merge(transactions[['hora','minuto','hora_do_dia','data', 'venda']]\
                                              .drop_duplicates('venda'), left_on=single_transaction.index, 
                                              right_on='venda', how='left', copy=False)

print('O número médio de itens vendidos por hora do dia é: \n{0}'.format(
    single_transaction.groupby(by='hora_do_dia')['item_count'].mean()))

O número médio de itens vendidos por hora do dia é: 
hora_do_dia
manhã    3.046419
noite    2.663510
tarde    2.783376
Name: item_count, dtype: float64


In [52]:
# Find the frequency of each item purchased and by time of day

item_frequencies_all = transactions.produto.value_counts()
item_frequencies_morning = transactions[transactions.hora_do_dia == 'manhã'].produto.value_counts()
item_frequencies_afternoon = transactions[transactions.hora_do_dia == 'tarde'].produto.value_counts()
item_frequencies_evening = transactions[transactions.hora_do_dia == 'noite'].produto.value_counts()

In [53]:
# Plot the frequency of purchase for each item

# https://community.plotly.com/t/errors-accessing-plotly-api/33271
# ok, so I found a solution to my problem - when I let my program run through, the error message produced is:
#chart_studio.exceptions.PlotlyRequestError: Authentication credentials were not provided.
#I Googled this error and there were suggestions to run plotly in offline mode by replacing:
#import chart_studio.plotly as py
#with

import plotly.offline as py
#then use py.plot instead of py.iplot

# Solving the problem of the 'plotly' online library with chart_studio
# https://plotly.com/python/getting-started-with-chart-studio/
import plotly.graph_objs as go
# import plotly.express as px

button_layer_1_height = 1.12
button_layer_2_height = 1.065

item_threshold_all = 100
item_threshold_other = 20

trace_all = go.Bar(
            x=item_frequencies_all[item_frequencies_all > item_threshold_all].index,
            y=item_frequencies_all[item_frequencies_all > item_threshold_all].values,
            visible=True,
            name='Todo dia'
    )

trace_morning = go.Bar(
            x=item_frequencies_morning[item_frequencies_morning > item_threshold_other].index,
            y=item_frequencies_morning[item_frequencies_morning > item_threshold_other].values,
            visible=False,
            name='Manhã'
    )

trace_afternoon = go.Bar(
            x=item_frequencies_afternoon[item_frequencies_afternoon > item_threshold_other].index,
            y=item_frequencies_afternoon[item_frequencies_afternoon > item_threshold_other].values,
            visible=False,
            name='Tarde'
    )

trace_evening = go.Bar(
            x=item_frequencies_evening[item_frequencies_evening > 10].index,
            y=item_frequencies_evening[item_frequencies_evening > 10].values,
            visible=False,
            name='Noite'
    )

data = [trace_all, trace_morning, trace_afternoon, trace_evening]

updatemenus = list([
    
    dict(
         buttons=list([   
            dict(label = 'Todo dia',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]},
                         {'title': 'Todo Dia'}]),
             
            dict(label = 'Manhã',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         {'title': 'Manhã'}]),
             
            dict(label = 'Tarde',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         {'title': 'Tarde'}]),
             
            dict(label = 'Noite',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         {'title': 'Noite'}])
        ]),
        type='buttons',
        direction = 'right',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.3,
        xanchor = 'left',
        y = button_layer_1_height,
        yanchor = 'top' )
])
                
layout = dict(title='Vendas Farina por hora do dia', showlegend=False,
              updatemenus=updatemenus)




fig = dict(data=data, layout=layout)
py.plot(fig, filename='Frequencia_vendas_hora_dia.html')

'Frequencia_vendas_hora_dia.html'

# Creating an animated frequencie line 

In [54]:
# Sales trend by date
# https://www.youtube.com/watch?v=Xjm7fOw6gjY
# use reset index for transforming the data in a variable not a index

grouped_by_date = pd.DataFrame(transactions.groupby(by='horario')['item_count'].sum().reset_index())

In [55]:
grouped_by_date.head()

,horario,item_count
0,2022-02-01,74
1,2022-02-02,88
2,2022-02-03,81
3,2022-02-04,110
4,2022-02-05,112


In [56]:
import plotly.figure_factory as ff

# Create a plot.ly-friendly table for Sales per day

table = ff.create_table(grouped_by_date)
py.plot(table, filename='tabela_vendas_por_dia_farina.html')


'tabela_vendas_por_dia_farina.html'

In [57]:
vendas = px.line(grouped_by_date, x = 'horario', y = 'item_count', title = 'Itens vendidos por dia', labels= {'item_count' : 'Quantidade de itens vendidos','horario': 'Data'})
py.plot(vendas, filename='Vendas_por_dia_linha.html')

'Vendas_por_dia_linha.html'

In [59]:
# ajustando o gráfio para retirar dois outliers (490 em 2022-07-30 , 2303 em 2022-06-23 ), utilizando-se de filtro com a função 'where'
indice_atip = np.where((grouped_by_date['item_count'] < 400))
linha=grouped_by_date.loc[indice_atip]

In [60]:
# https://towardsdatascience.com/line-chart-animation-with-plotly-on-jupyter-e19c738dc882



#linha=grouped_by_date: usar se precisar demonstrar os dois outliers

trace1 = go.Scatter(x=linha['horario'][:2],
                    y=linha['item_count'][:2],
                    mode='lines',
                    line=dict(width=1.5),
                    fill = 'tozeroy',
                   fillcolor = 'lightblue'
)
frames = [dict(data= [dict(type='scatter',
                           x=linha['horario'][:k+1],
                           y=linha['item_count'][:k+1]),
                     ],
               traces= [0],  
              )for k  in  range(1, len(linha)-1)]
layout = go.Layout(width=1180,
                   height=750,
                   showlegend=False,
                   hovermode='closest',
                   updatemenus=[
                        dict(
                            type='buttons', showactive=False,
                            y=1.06,
                            x=0.12,
                            xanchor='center',
                            yanchor='auto',
                            pad=dict(t=0, r=10),
                            buttons=[dict(label='Play',
                            method='animate',
                            args=[None, 
                                  dict(frame=dict(duration=30, 
                                                  redraw=False),
                                                  transition=dict(duration=0),
                                                  fromcurrent=True,
                                                  mode='immediate')]
                            )]
                        ),
                        
                    ]              
                  )
layout.update(xaxis =dict(range=['2022-02-01', '2022-10-31'], autorange=False),
              yaxis =dict(range=[0, 250], autorange=False)); # ajustar o range se precisar colocar os outliers
fig_vendas = go.Figure(data=[trace1], frames=frames, layout=layout)

py.plot(fig_vendas, filename='Vendas_por_dia_farina_interativo.html')

'Vendas_por_dia_farina_interativo.html'

# Investigando dados atipicos

In [61]:
# Vendas atípicas no dia 23 de Junho de 2022 entre 16:00 e 17:00

df_invest = pd.DataFrame(transactions).query("data == 'Jun 23, 2022'").groupby(by='hora')['item_count'].sum().reset_index()


In [62]:
dataframe_to_pdf(df_invest, "investiga_dados_atipicos.pdf", numpages=(1, 1), pagesize=(11.7, 8.3))

In [67]:
graf = px.line(df_invest, x = 'hora', y = 'item_count', labels= {'item_count':'Itens Vendidos'}, log_y= True,
title= 'Investigando dados atipicos - qantidade itens vendidos no dia 23 de Junho 2022')

py.plot(graf, filename='investiga_graf.html')


'investiga_graf.html'

# Vendas dias de semana (DDS) vs Final de semana (FDS)

In [68]:
dds_vendas= pd.DataFrame(transactions[transactions.tipo_de_dia == 'DDS'].groupby(by='horario')['item_count'].sum())
fds_vendas= pd.DataFrame(transactions[transactions.tipo_de_dia == 'FDS'].groupby(by='horario')['item_count'].sum())

In [69]:
dds_vendas_filter = (dds_vendas['item_count']< 399)

In [70]:
fds_vendas_filter = (fds_vendas['item_count'] < 399)

# Observar esses dados

In [71]:
print('O menor número de vendas em um dia de semana foi: \n{0}'.format(
    dds_vendas.item_count.min()))
print('O número médio de vendas em um dia de semana é: \n{0}'.format(
    dds_vendas.item_count.mean()))
print('O maior número de vendas em um dia de semana foi: \n{0}'.format(
    dds_vendas.item_count.max()))

O menor número de vendas em um dia de semana foi: 
1
O número médio de vendas em um dia de semana é: 
88.18333333333334
O maior número de vendas em um dia de semana foi: 
2303


In [72]:
print('O menor número de vendas em um dia de fim-de-semana foi: \n{0}'.format(
    fds_vendas.item_count.min()))
print('O número médio de vendas em um dia fim-de-semana é: \n{0}'.format(
    fds_vendas.item_count.mean()))
print('O maior número de vendas em um dia fim-de-semana foi: \n{0}'.format(
    fds_vendas.item_count.max()))

O menor número de vendas em um dia de fim-de-semana foi: 
65
O número médio de vendas em um dia fim-de-semana é: 
138.6164383561644
O maior número de vendas em um dia fim-de-semana foi: 
490


In [73]:
single_transaction = pd.DataFrame(transactions.groupby(by='venda')['item_count'].sum())

single_transaction = single_transaction.merge(transactions[['tipo_de_dia','horario', 'venda']]\
                                              .drop_duplicates('venda'), left_on=single_transaction.index, 
                                              right_on='venda', how='left', copy=False)

print('O número médio de itens em cada venda por tipo de dia é: \n{0}'.format(
    single_transaction.groupby(by='tipo_de_dia')['item_count'].mean()))

O número médio de itens em cada venda por tipo de dia é: 
tipo_de_dia
DDS    2.768707
FDS    2.828116
Name: item_count, dtype: float64


## Itens vendidos por tipo de dia

In [76]:
# Histogram of items sold by type of day

trace_dds = go.Histogram(
    x=dds_vendas[dds_vendas.item_count < 200].item_count,
    opacity=0.75,
    name = 'Dia de Semana'
)

trace_fds = go.Histogram(
    x=fds_vendas[fds_vendas.item_count < 399].item_count,
    opacity=0.65,
    name = 'Fim-de-semana',
    nbinsx = 16
)

data = [trace_dds, trace_fds]
layout = go.Layout( title = 'Distribuição das vendas por dias de semana/ finais de semana',
                   xaxis=dict(title='Número de Itens Vendidos'),
                   yaxis=dict(title='Frequencia de vendas'),
                   barmode='overlay'
                   )

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='tipo-de_dia_itens_histograma.html')

'tipo-de_dia_itens_histograma.html'

# Faturamento por tipo de dia (DDS x FDS)

In [106]:
dds_fatu= pd.DataFrame(transactions[transactions.tipo_de_dia == 'DDS'].groupby(by='horario')['total_item'].sum())
fds_fatu= pd.DataFrame(transactions[transactions.tipo_de_dia == 'FDS'].groupby(by='horario')['total_item'].sum())

In [110]:
# Histograma

trace_dds = go.Histogram(
    x=dds_fatu[dds_fatu.total_item  < 7900].total_item,
    opacity=0.75,
    name = 'Dia de Semana'
)

trace_fds = go.Histogram(
    x=fds_fatu[fds_fatu.total_item < 7900].total_item,
    opacity=0.65,
    name = 'Fim-de-semana',
    nbinsx = 16
)

data = [trace_dds, trace_fds]
layout = go.Layout( title = 'Distribuição do faturamento por dias de semana/ finais de semana',
                   xaxis=dict(title='Faturamento por dia'),
                   yaxis=dict(title='Frequencia de faturamento diário'),
                   barmode='overlay'
                   )

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='tipo-de_dia_faturamento_histograma.html')

'tipo-de_dia_faturamento_histograma.html'

# Número de itens vendidos por dia da semana

In [111]:
f_filter = (transactions['data']!='2022-06-23')&(transactions['data']!='2022-07-30')
transactions[f_filter]

transactions_filtrada = transactions[f_filter]

In [112]:
grouped_by_date_filtrada = pd.DataFrame(transactions_filtrada.groupby(by='horario')['item_count'].sum())

In [113]:
grouped_by_date_filtrada.head()

,item_count
horario,
2022-02-01,74
2022-02-02,88
2022-02-03,81
2022-02-04,110
2022-02-05,112


In [114]:
# Find the number of itens sold per day of week

agregado_vendas_por_dia = grouped_by_date_filtrada.merge(transactions[['dia_da_semana', 'horario']], left_index=True, right_on='horario',
        how= 'left', validate='1:m')



In [115]:
agregado_vendas_por_dia.head()

,item_count,dia_da_semana,horario
0,74,terça-feira,2022-02-01
1,74,terça-feira,2022-02-01
2,74,terça-feira,2022-02-01
3,74,terça-feira,2022-02-01
4,74,terça-feira,2022-02-01


In [116]:
agregado_vendas_por_dia = agregado_vendas_por_dia[~agregado_vendas_por_dia.index.duplicated(keep='first')]

dias = ['segunda-feira','terça-feira','quarta-feira','quinta-feira','sexta-feira','sábado','domingo']

vendas_por_dia = []

for dia in dias:
    
    df_dias= agregado_vendas_por_dia[agregado_vendas_por_dia.dia_da_semana == dia]
    
    vendas_por_dia.append(list(df_dias['item_count']))
    


In [119]:
traces = []

for i, j in enumerate(dias):
    
    trace = go.Box(
    y=vendas_por_dia[i],
    name = j
    )
    traces.append(trace)

data = traces

layout = go.Layout(title='Vendas por dia da semana',
                  xaxis = dict(title = 'Dia da Semana'),
                  yaxis = dict(title = 'Número de Itens Vendidos'))

fig = go.Figure(data = data, layout = layout)

py.plot(fig, filename='Vendas_por_dia_da_semana_Boxplot.html')

'Vendas_por_dia_da_semana_Boxplot.html'

# Horário de vendas dos itens por dia da semana

In [124]:
dia_hora_df = transactions_filtrada[[ 'hora_minuto','dia_da_semana', 'hora', 'item_count']]


### Modo Simples

In [94]:
fig1 = px.box(dia_hora_df, y = 'hora', hover_data=['item_count'])

fig1.show()

In [126]:
fig = px.box(dia_hora_df, y = 'hora', x = 'dia_da_semana', color='dia_da_semana', labels={'hora': 'Hora','dia_da_semana':'Dia da Semana'},
title="Hora das vendas dia da semana", hover_data = [dia_hora_df.hora_minuto], points="all", 
category_orders= {"dia_da_semana": ['sábado','domingo','segunda-feira','terça-feira','quarta-feira','quinta-feira','sexta-feira']})

import plotly
py.plot(fig, filename='Hora_vendas_dias_semana.html')

'Hora_vendas_dias_semana.html'

In [96]:
# https://stackoverflow.com/questions/59815797/how-to-save-plotly-express-plot-into-a-html-or-static-image-file
# fig.write_html(r"C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\Vendas_por_dia_Boxplot_simples2.html")


### Modo Complexo - Graphic Objects

In [129]:
dia_hora_df.head()

,hora_minuto,dia_da_semana,hora,item_count
0,08:13:00,terça-feira,8,1
1,08:13:00,terça-feira,8,1
2,08:14:00,terça-feira,8,1
3,08:14:00,terça-feira,8,1
4,09:01:00,terça-feira,9,1


### How to use Graphics Objects


In [130]:
#  https://www.youtube.com/watch?v=AwtQ7vJg0-o

In [136]:
dom = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'domingo']
seg = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'segunda-feira']
ter = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'terça-feira']
qua = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'quarta-feira']
qui = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'quinta-feira']
sex = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'sexta-feira']
sab = dia_hora_df.hora[dia_hora_df['dia_da_semana']== 'sábado']

In [141]:
trace0 = go.Box(
    y= dom, boxpoints = 'all', jitter = 0.5,
     name = 'Domingo',
)

trace1 = go.Box(
    y= seg,
    name = 'Segunda'
)

trace2 = go.Box(
    y= ter,
    name = 'Terça'
)

trace3 = go.Box(
    y= qua,
    name = 'Quarta'
)

trace4 = go.Box(
    y= qui,
    name = 'Quinta'
)

trace5 = go.Box(
    y= sex,
    name = 'Sexta'
)

trace6 = go.Box(
    y= sab,
    name = 'Sábado'
)

In [143]:
data = [trace6, trace0, trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(title= 'Vendas por hora e dia da semana')

In [144]:
fig = go.Figure(data = data, layout = layout)


In [145]:
py.iplot(fig, filename='Vendas_hora_por_dia_Boxplot.html')

## Quantidade de itens vendidos por dia da semana e período


In [105]:
group_by_data_e_hora_do_dia = transactions_filtrada.groupby(by=['dia_da_semana','hora_do_dia'])['item_count'].sum()

In [106]:
# https://stackoverflow.com/questions/39275294/sort-by-certain-order-situation-pandas-dataframe-groupby
# #### Alterando a ordem de apresentação dos dias da semana(Seg - Dom), usando a lista 'dias' anteriormente criada

group_by_data_e_hora_do_dia = group_by_data_e_hora_do_dia.reindex(dias, level= 'dia_da_semana')

In [107]:
group_by_data_e_hora_do_dia = pd.DataFrame(group_by_data_e_hora_do_dia).reset_index()

In [108]:
group_by_data_e_hora_do_dia

,dia_da_semana,hora_do_dia,item_count
0,segunda-feira,manhã,587
1,segunda-feira,noite,1354
2,segunda-feira,tarde,548
3,terça-feira,manhã,545
4,terça-feira,noite,1359
5,terça-feira,tarde,619
6,quarta-feira,manhã,642
7,quarta-feira,noite,1403
8,quarta-feira,tarde,811
9,quinta-feira,manhã,526


In [109]:
group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'manhã'].item_count

0      587
3      545
6      642
9      526
12     744
15    1512
18    2310
Name: item_count, dtype: int64

In [110]:
trace_manhã = go.Bar(
    y = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'manhã'].item_count,
    x = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'manhã'].dia_da_semana,
    name = "manhã",
    opacity=0.6,
)

trace_tarde = go.Bar(
   y = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'tarde'].item_count,
    x = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'tarde'].dia_da_semana,
    name = "tarde",
    opacity=0.6
)

trace_noite = go.Bar(
   y = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'noite'].item_count,
    x = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'noite'].dia_da_semana,
    name = "noite",
    opacity=0.6
)

layout = go.Layout(title='Vendas por período e dia da semana',
                  xaxis = dict(title = 'Dia da Semana'),
                  yaxis = dict(title = 'Número de Itens Vendidos'), hovermode= 'x')

data = [trace_manhã, trace_tarde, trace_noite]

fig = go.Figure(data = data, layout = layout)  

py.iplot(fig, filename='vendas-por-dia-periodo')

### Vendas por hora

In [111]:
vendas_por_hora = pd.DataFrame(
                transactions_filtrada[(transactions_filtrada.hora > 1) & 
                (transactions_filtrada.hora < 21)].groupby(by='hora')['item_count'].sum())

In [112]:
vendas_por_hora

,item_count
hora,
6,41
7,524
8,1289
9,1749
10,1817
11,1446
12,737
13,627
14,769


In [113]:
trace = go.Scatter(x=list(vendas_por_hora.index),
                   y=list(vendas_por_hora.item_count),
                   fill = 'tozeroy',
                   fillcolor = 'lightblue')

data = [trace]

layout = dict(
    title='Total de volume de vendas por hora',
    yaxis = dict( title = 'Volume de Itens Vendidos'),
    xaxis=dict( title = 'Hora',
        rangeselector=dict(
            buttons=list([
                dict(count = 12,
                     label='Manhã',
                     step='hour',
                     stepmode='backward'),
                dict(count = 8,
                     label='Tarde',
                     step='hour',
                     stepmode='backward'),
                dict(count = 4,
                    label='Noite',
                    step='hour',
                    stepmode='backward'),
                dict(step='all')
            ]),
            visible = True
        ),
        rangeslider=dict(
            visible = True
        )
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename = 'vendas-por-hora')

# Apriori Algorithm, Method 1

In [114]:
# Create a list of lists that is conducive to the format expected from apyori

list_of_lists = []

for transaction in list(set(transactions_filtrada.venda)):
    
    df = transactions_filtrada[transactions_filtrada.venda == transaction]
    values = df.produto.values
    
    list_of_lists.append(list(values))

In [115]:
# imprting the 'apriori' function from the 'apyori' module
from apyori import apriori

# Create a list of association rules
association_rules = apriori(list_of_lists, min_confidence = 0.2, min_support = 0.005,min_lift = 2, min_length=2)  
association_results = list(association_rules)

In [116]:
print(association_results)

[RelationRecord(items=frozenset({'ADICIONAL DE PRESUNTO', 'ADICIONAL DE MUSSARELA'}), support=0.0061068702290076335, ordered_statistics=[OrderedStatistic(items_base=frozenset({'ADICIONAL DE MUSSARELA'}), items_add=frozenset({'ADICIONAL DE PRESUNTO'}), confidence=0.5416666666666666, lift=70.95833333333333), OrderedStatistic(items_base=frozenset({'ADICIONAL DE PRESUNTO'}), items_add=frozenset({'ADICIONAL DE MUSSARELA'}), confidence=0.8, lift=70.95833333333333)]), RelationRecord(items=frozenset({'CROISSANT', 'ADICIONAL DE MUSSARELA'}), support=0.009395184967704051, ordered_statistics=[OrderedStatistic(items_base=frozenset({'ADICIONAL DE MUSSARELA'}), items_add=frozenset({'CROISSANT'}), confidence=0.8333333333333333, lift=4.038607474862455)]), RelationRecord(items=frozenset({'CROISSANT', 'ADICIONAL DE PRESUNTO'}), support=0.006928948913681738, ordered_statistics=[OrderedStatistic(items_base=frozenset({'ADICIONAL DE PRESUNTO'}), items_add=frozenset({'CROISSANT'}), confidence=0.9076923076923

In [117]:
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")


Rule: ADICIONAL DE PRESUNTO -> ADICIONAL DE MUSSARELA
Support: 0.0061068702290076335
Confidence: 0.5416666666666666
Lift: 70.95833333333333
Rule: CROISSANT -> ADICIONAL DE MUSSARELA
Support: 0.009395184967704051
Confidence: 0.8333333333333333
Lift: 4.038607474862455
Rule: CROISSANT -> ADICIONAL DE PRESUNTO
Support: 0.006928948913681738
Confidence: 0.9076923076923077
Lift: 4.398975526465566
Rule: CROISSANT -> AGUA MINERAL S/GAS 500ML
Support: 0.01996476805637111
Confidence: 0.4197530864197531
Lift: 2.0342615428936814
Rule: PAO SOURDOUGH OVOS -> BOWL DE IOGURTE FRUTAS AVEIA
Support: 0.007868467410452143
Confidence: 0.24814814814814815
Lift: 9.107678799489145
Rule: CAFE CATUAI VERMELHO C/LEITE G -> CROISSANT
Support: 0.007868467410452143
Confidence: 0.43506493506493504
Lift: 2.1084677985645546
Rule: CAFE CATUAI VERMELHO G 200ML -> CROISSANT
Support: 0.013740458015267175
Confidence: 0.46799999999999997
Lift: 2.2680819578827545
Rule: CROISSANT -> CAFE COLMEIA C/ LEITE G 200ML
Support: 0.005

## Apriori Algorithm, Method 1 - Manhã

In [118]:
manha_filter = (transactions_filtrada['hora_do_dia']=='manhã')
transactions_filtrada_manha = transactions_filtrada[manha_filter]
# Create a list of lists that is conducive to the format expected from apyori


list_of_lists_manha = []

for transaction in list(set(transactions_filtrada_manha.venda)):
    
    df = transactions_filtrada_manha[transactions_filtrada_manha.venda == transaction]
    values = df.produto.values
    
    list_of_lists_manha.append(list(values))

In [119]:
# imprting the 'apriori' function from the 'apyori' module
from apyori import apriori

# Create a list of association rules
association_rules_manha = apriori(list_of_lists_manha, min_confidence = 0.2, min_support = 0.005,min_lift = 3, min_length=3)  
association_results_manha = list(association_rules_manha)

In [120]:
association_results_manha

[RelationRecord(items=frozenset({'ADICIONAL DE PRESUNTO', 'ADICIONAL DE MUSSARELA'}), support=0.009764758100310697, ordered_statistics=[OrderedStatistic(items_base=frozenset({'ADICIONAL DE MUSSARELA'}), items_add=frozenset({'ADICIONAL DE PRESUNTO'}), confidence=0.5945945945945946, lift=40.5945945945946), OrderedStatistic(items_base=frozenset({'ADICIONAL DE PRESUNTO'}), items_add=frozenset({'ADICIONAL DE MUSSARELA'}), confidence=0.6666666666666667, lift=40.594594594594604)]),
 RelationRecord(items=frozenset({'AGUA MINERAL S/GAS 500ML', 'ADICIONAL DE MUSSARELA'}), support=0.005326231691078562, ordered_statistics=[OrderedStatistic(items_base=frozenset({'ADICIONAL DE MUSSARELA'}), items_add=frozenset({'AGUA MINERAL S/GAS 500ML'}), confidence=0.3243243243243244, lift=5.294947121034079)]),
 RelationRecord(items=frozenset({'CROISSANT', 'ADICIONAL DE MUSSARELA'}), support=0.015090989791389259, ordered_statistics=[OrderedStatistic(items_base=frozenset({'ADICIONAL DE MUSSARELA'}), items_add=froz

In [121]:
for item in association_results_manha:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: ADICIONAL DE PRESUNTO -> ADICIONAL DE MUSSARELA
Support: 0.009764758100310697
Confidence: 0.5945945945945946
Lift: 40.5945945945946
Rule: AGUA MINERAL S/GAS 500ML -> ADICIONAL DE MUSSARELA
Support: 0.005326231691078562
Confidence: 0.3243243243243244
Lift: 5.294947121034079
Rule: CROISSANT -> ADICIONAL DE MUSSARELA
Support: 0.015090989791389259
Confidence: 0.918918918918919
Lift: 3.344627341396324
Rule: CROISSANT -> ADICIONAL DE PRESUNTO
Support: 0.013315579227696404
Confidence: 0.9090909090909091
Lift: 3.3088559259803203
Rule: AGUA MINERAL SEM GAS 500 ML -> CROISSANT BRIE PARMA MEL DE
Support: 0.007545494895694629
Confidence: 0.2
Lift: 5.006666666666667
Rule: PAO SOURDOUGH OVOS -> BOWL DE IOGURTE FRUTAS AVEIA
Support: 0.024411895250776743
Confidence: 0.2972972972972973
Lift: 3.9170222854433385
Rule: BOWL DE IOGURTE FRUTAS AVEIA -> SELECAO DE PAES GELEIA
Support: 0.010208610741233911
Confidence: 0.2674418604651163
Lift: 3.2570081709616594
Rule: CAFE CATUAI VERMELHO C/LEITE G -> LA

In [122]:
rule_list_manha = []
for item in association_results_manha:
    
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]

    rule_list_manha.append ( items[0] + " -> " + items[1])

support_list_manha =[]
for item in association_results_manha:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    #items = [x for x in pair]
    

    #second index of the inner list
    support_list_manha.append( str(item[1]))

confidence_list_manha =[]
for item in association_results_manha:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    

    #second index of the inner list
    confidence_list_manha.append(str(item[2][0][2]))

lift_list_manha =[]
for item in association_results_manha:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    
    #second index of the inner list
    lift_list_manha.append(str(item[2][0][3]))

In [123]:
association_results_manha_df=pd.DataFrame(list
(zip(rule_list_manha, support_list_manha, confidence_list_manha, lift_list_manha)), columns=['Rule','Support', 'Confidence', 'Lift'])

In [124]:
association_results_manha_df.sort_values(by=['Support'],ascending=False)

,Rule,Support,Confidence,Lift
5,PAO SOURDOUGH OVOS -> BOWL DE IOGURTE FRUTAS A...,0.024411895250776743,0.2972972972972973,3.9170222854433385
2,CROISSANT -> ADICIONAL DE MUSSARELA,0.015090989791389259,0.918918918918919,3.344627341396324
3,CROISSANT -> ADICIONAL DE PRESUNTO,0.013315579227696404,0.9090909090909091,3.3088559259803203
6,BOWL DE IOGURTE FRUTAS AVEIA -> SELECAO DE PAE...,0.010208610741233911,0.2674418604651163,3.2570081709616594
0,ADICIONAL DE PRESUNTO -> ADICIONAL DE MUSSARELA,0.009764758100310697,0.5945945945945946,40.5945945945946
16,CROISSANT -> ADICIONAL DE PRESUNTO,0.009320905459387484,0.5675675675675677,42.624324324324334
10,SUCO DE LARANJA 400ML -> CROISSANT BRIE PARMA ...,0.00887705281846427,0.22222222222222224,6.032128514056224
14,CROISSANT OVOS BACON MEL -> SUCO DE LARANJA 400ML,0.008433200177541057,0.2289156626506024,5.157469879518072
12,CROISSANT IFOOD -> SOURDOUGH 369G IFOOD,0.008433200177541057,0.59375,7.309938524590163
11,CROISSANT IFOOD -> PASTEL DE NATA 60G IFOOD,0.008433200177541057,0.5428571428571428,6.683372365339578


# Convert a list to a Dataframe

In [125]:
rule_list = []
for item in association_results:
    
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]

    rule_list.append ( items[0] + " -> " + items[1])

support_list =[]
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    #items = [x for x in pair]
    

    #second index of the inner list
    support_list.append( str(item[1]))
confidence_list =[]
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    

    #second index of the inner list
    confidence_list.append(str(item[2][0][2]))
lift_list =[]
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    
    #second index of the inner list
    lift_list.append(str(item[2][0][3]))

In [126]:
association_results_df=pd.DataFrame(list
(zip(rule_list, support_list, confidence_list, lift_list)), columns=['Rule','Support', 'Confidence', 'Lift'])

In [127]:
association_results_df

,Rule,Support,Confidence,Lift
0,ADICIONAL DE PRESUNTO -> ADICIONAL DE MUSSARELA,0.0061068702290076335,0.5416666666666666,70.95833333333333
1,CROISSANT -> ADICIONAL DE MUSSARELA,0.009395184967704051,0.8333333333333333,4.038607474862455
2,CROISSANT -> ADICIONAL DE PRESUNTO,0.006928948913681738,0.9076923076923077,4.398975526465566
3,CROISSANT -> AGUA MINERAL S/GAS 500ML,0.01996476805637111,0.4197530864197531,2.0342615428936814
4,PAO SOURDOUGH OVOS -> BOWL DE IOGURTE FRUTAS A...,0.007868467410452143,0.24814814814814815,9.107678799489145
5,CAFE CATUAI VERMELHO C/LEITE G -> CROISSANT,0.007868467410452143,0.43506493506493504,2.1084677985645546
6,CAFE CATUAI VERMELHO G 200ML -> CROISSANT,0.013740458015267175,0.46799999999999997,2.2680819578827545
7,CROISSANT -> CAFE COLMEIA C/ LEITE G 200ML,0.00551967116852613,0.5280898876404494,2.5592973211487915
8,CAFE COLMEIA G 200ML -> CROISSANT,0.007633587786259542,0.4276315789473684,2.0724433094688917
9,CROISSANT -> LARANJA 400 ML,0.021374045801526718,0.6275862068965518,3.041489215551587


In [165]:
table_2 = ff.create_table(association_results_df, height_constant=40)
py.plot(table_2, filename='Apriori_modelo_01.html')

'Apriori_modelo_01.html'

# Resultado inconsistente

In [129]:
def apriori_results(min_confidence=0, min_support=0.005, sort_by_value = 'lift', min_length =None,
                    hora_do_dia=None, dia_da_semana = None, df=transactions_filtrada):
    
    """
    Takes in a data frame, filters it by time of day, calculates the results of the Apriori algorithm,
    and returns a data frame sorted by lift.
    """
    
    from apyori import apriori
    
    if hora_do_dia != None:
        
        df = df[df.hora_do_dia == hora_do_dia]
        
    if dia_da_semana:
        
        df = df[df.tipo_de_dia == dia_da_semana]
        
    list_of_lists = []

    for transaction in list(set(df.venda)):

        df_t = df[df.venda == transaction]
        values = df_t.produto.values

        list_of_lists.append(list(values))

    if min_length is not None:
        
        list_of_lists = [i for i in list_of_lists if len(i) >= min_length]
        
    association_rules = apriori(list_of_lists, min_confidence = min_confidence, min_support = min_support)  
    association_results = list(association_rules)

    item_1 = []
    item_2 = []
    support = []
    confidence = []
    lift = []

    for item in association_results:

        pair = item[0] 
        items = [x for x in pair]
        
        if min_length is not None:
            
            if len(items) < min_length:

                continue

            else:

                item_1.append(items[0])

            #try:

                item_2.append(items[1])

            #except:

            #    item_2.append('None')
            
        else:
            
            item_1.append(items[0])

            try:

                item_2.append(items[1])

            except:

                item_2.append('None')

        support.append(item[1])

        confidence.append(item[2][0][2])
        lift.append(item[2][0][3])

    df_apriori = pd.DataFrame({'item_1':item_1, 'item_2':item_2, 'support':support,'confidence':confidence,'lift':lift})
    
    df_apriori = df_apriori.sort_values(by=sort_by_value, ascending=False)
    
    return df_apriori

In [130]:
all_day_results = apriori_results()

In [131]:
all_day_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.007986,0.007986,1.0
104,CROISSANT,BOWL DE IOGURTE FRUTAS AVEIA,0.008103,0.008103,1.0
106,CAFE CATUAI VERMELHO C/LEITE G,CROISSANT,0.007868,0.007868,1.0
107,CAFE CATUAI VERMELHO G 200ML,CROISSANT,0.013740,0.013740,1.0
108,CAFE CATUAI VERMELHO G 200ML,PASTEL DE NATA 60G,0.005872,0.005872,1.0
109,CROISSANT,CAFE COLMEIA C/ LEITE G 200ML,0.005520,0.005520,1.0
110,CAFE COLMEIA G 200ML,CROISSANT,0.007634,0.007634,1.0
111,CROISSANT,CAFE EQUILIBRADO CAPSULA G,0.005167,0.005167,1.0
112,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.009043,0.009043,1.0
113,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.010570,0.010570,1.0


In [132]:
morning_results = apriori_results(hora_do_dia ='manhã')

In [133]:
morning_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.009321,0.009321,1.0
153,COOKIES 80G,CROISSANT,0.016866,0.016866,1.0
142,CAFE EQUILIBRADO CAPSULA G,PAO SOURDOUGH OVOS,0.007102,0.007102,1.0
143,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.011540,0.011540,1.0
144,CAFE EQUILIBRADO CAPSULA P,CROISSANT OVOS BACON MEL,0.005770,0.005770,1.0
145,CAFE EQUILIBRADO CAPSULA P,PAO SOURDOUGH OVOS,0.005326,0.005326,1.0
146,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.008433,0.008433,1.0
147,CROISSANT,CAFE INTENSO CAPSULA G 150ML,0.009321,0.009321,1.0
148,PAO SOURDOUGH OVOS,CAFE INTENSO CAPSULA G 150ML,0.005770,0.005770,1.0
149,CROISSANT,CAFE INTENSO CAPSULA P 50ML,0.007989,0.007989,1.0


In [134]:
afternoon_results = apriori_results(hora_do_dia ='tarde')

In [135]:
afternoon_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.011337,0.011337,1.0
1,ADICIONAL DE MUSSARELA,None,0.010392,0.010392,1.0
102,CAFE EQUILIBRADO CAPSULA G,PASTEL DE NATA 60G,0.005668,0.005668,1.0
103,CAFE EQUILIBRADO CAPSULA P,CAFE INTENSO CAPSULA P 50ML,0.007085,0.007085,1.0
104,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.007558,0.007558,1.0
105,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.014643,0.014643,1.0
106,CROISSANT,CAFE INTENSO CAPSULA P 50ML,0.005668,0.005668,1.0
107,PASTEL DE NATA 60G,CAFE INTENSO CAPSULA P 50ML,0.007085,0.007085,1.0
108,CROISSANT,COCA COLA LATA 350 ML,0.011809,0.011809,1.0
109,FOLHADO FRANGO 160G,COCA COLA LATA 350 ML,0.005196,0.005196,1.0


In [136]:
evening_results = apriori_results(hora_do_dia='noite')

In [137]:
evening_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.005549,0.005549,1.0
122,CROISSANT,FOLHADO FRANGO 160G,0.012545,0.012545,1.0
104,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.009650,0.009650,1.0
105,CROISSANT,COCA COLA LATA 350 ML,0.012063,0.012063,1.0
106,CROISSANT BRIE PARMA MEL DE,COCA COLA LATA 350 ML,0.005549,0.005549,1.0
107,FOLHADO FRANGO 160G,COCA COLA LATA 350 ML,0.005790,0.005790,1.0
108,FOLHADO MACA 80G,COCA COLA LATA 350 ML,0.007238,0.007238,1.0
109,PASTEL DE NATA 60G,COCA COLA LATA 350 ML,0.009168,0.009168,1.0
110,PIZZA AL TAGLIO GORGONZOLA E,COCA COLA LATA 350 ML,0.006996,0.006996,1.0
111,SANDUICHE DE PEPERONI BRIE,COCA COLA LATA 350 ML,0.005066,0.005066,1.0


In [138]:
weekday_results = apriori_results(dia_da_semana= 'DDS')

In [139]:
weekday_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.007812,0.007812,1.0
94,CROISSANT,BOMBOLONI DOCE DE LEITE 80G,0.007422,0.007422,1.0
96,CROISSANT,BOMBOLONI NUTELLA 80G,0.009570,0.009570,1.0
97,FOLHADO FRANGO 160G,BOMBOLONI NUTELLA 80G,0.005469,0.005469,1.0
98,BOMBOLONI NUTELLA 80G,PASTEL DE NATA 60G,0.007227,0.007227,1.0
99,PAO SOURDOUGH OVOS,BOWL DE IOGURTE FRUTAS AVEIA,0.006836,0.006836,1.0
100,CAFE CATUAI VERMELHO C/LEITE G,CROISSANT,0.006055,0.006055,1.0
101,CAFE CATUAI VERMELHO G 200ML,CROISSANT,0.011914,0.011914,1.0
102,CROISSANT,CAFE COLMEIA C/ LEITE G 200ML,0.005859,0.005859,1.0
103,CAFE COLMEIA G 200ML,CROISSANT,0.008789,0.008789,1.0


In [140]:
weekend_results = apriori_results(dia_da_semana = 'FDS')

In [141]:
weekend_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.008247,0.008247,1.0
131,COOKIES 80G,CROISSANT AMENDOAS 100G,0.006186,0.006186,1.0
122,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.011193,0.011193,1.0
123,CAFE EQUILIBRADO CAPSULA P,CROISSANT AMENDOAS 100G,0.005302,0.005302,1.0
124,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.011487,0.011487,1.0
125,CROISSANT,CAFE INTENSO CAPSULA P 50ML,0.005007,0.005007,1.0
126,PASTEL DE NATA 60G,CAFE INTENSO CAPSULA P 50ML,0.005007,0.005007,1.0
127,CROISSANT,COCA COLA LATA 350 ML,0.010015,0.010015,1.0
128,PASTEL DE NATA 60G,COCA COLA LATA 350 ML,0.007069,0.007069,1.0
129,COCA COLA ZERO LATA 350 ML,CROISSANT,0.008542,0.008542,1.0


In [142]:
weekday_morning_results = apriori_results(dia_da_semana = 'DDS', hora_do_dia = 'manhã')

In [143]:
weekday_morning_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.009506,0.009506,1.0
148,CROISSANT OVOS BACON MEL,CAFE INTENSO CAPSULA P 50ML,0.005703,0.005703,1.0
137,CAFE EQUILIBRADO CAPSULA G,PAO SOURDOUGH OVOS,0.009506,0.009506,1.0
138,CAFE EQUILIBRADO CAPSULA G,PASTEL DE NATA 60G,0.005703,0.005703,1.0
139,CAFE EQUILIBRADO CAPSULA P,CAFE INTENSO CAPSULA P 50ML,0.005703,0.005703,1.0
140,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.014259,0.014259,1.0
141,CAFE EQUILIBRADO CAPSULA P,CROISSANT OVOS BACON MEL,0.007605,0.007605,1.0
142,CAFE EQUILIBRADO CAPSULA P,FOLHADO FRANGO 160G,0.005703,0.005703,1.0
143,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.009506,0.009506,1.0
144,CROISSANT,CAFE INTENSO CAPSULA G 150ML,0.013308,0.013308,1.0


In [144]:
weekday_afternoon_results = apriori_results(dia_da_semana = 'DDS', hora_do_dia = 'tarde')

In [145]:
weekday_afternoon_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.011021,0.011021,1.0
119,COOKIES 80G,FOLHADO FRANGO 160G,0.005143,0.005143,1.0
111,CROISSANT,COCA COLA LATA 350 ML,0.011756,0.011756,1.0
112,FOLHADO FRANGO 160G,COCA COLA LATA 350 ML,0.005143,0.005143,1.0
113,PASTEL DE NATA 60G,COCA COLA LATA 350 ML,0.005878,0.005878,1.0
114,SANDUICHE DE PEPERONI BRIE,COCA COLA LATA 350 ML,0.007348,0.007348,1.0
115,COOKIES 80G,CROISSANT,0.016899,0.016899,1.0
116,COOKIES 80G,CROISSANT AMENDOAS 100G,0.005143,0.005143,1.0
117,COOKIES 80G,CROISSANT TRADICIONAL 80 G,0.005143,0.005143,1.0
118,COOKIES 80G,FOCACCIA FORMAGGIO 246G,0.006613,0.006613,1.0


# Apriori Algorithm, Method 2

In [146]:
def apriori_results_method_2(df = transactions_filtrada, min_support = 0.005, hora_do_dia = None, dia_da_semana = None,
                            min_threshold = 1, metric = 'lift'):
    
    """
    Takes in a data frame and returns the results of the Apriori algorithm using the mlxtend Python package.
    """
    
    from mlxtend.frequent_patterns import apriori
    from mlxtend.frequent_patterns import association_rules

    if hora_do_dia:
        
        df = df[df.hora_do_dia == hora_do_dia]
        
    if dia_da_semana:
        
        df = df[df.tipo_de_dia == dia_da_semana]
        
    dummies_df = df.groupby(['venda', 'produto'])['produto'].count().unstack().\
                 reset_index().fillna(0).set_index('venda')
    
    def encode_units(x):
    
        if x <= 0:
            return 0
        if x >= 1:
            return 1
    
    dummies_df = dummies_df.applymap(encode_units)
    
    frequent_itemsets = apriori(dummies_df, min_support= min_support, use_colnames=True)

    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold)
    
    rules.drop_duplicates(subset='leverage', keep='first', inplace=True)
    rules = rules.sort_values(by='lift', ascending=False)
    
    return rules

In [147]:
all_day_method_2 = apriori_results_method_2()

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [148]:
all_day_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
115,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.009395,0.007634,0.005637,0.600000,78.600000,0.005565,2.480916
114,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.006929,0.011274,0.005637,0.813559,72.161017,0.005559,5.303166
0,(ADICIONAL DE PRESUNTO),(ADICIONAL DE MUSSARELA),0.007634,0.011274,0.006107,0.800000,70.958333,0.006021,4.943629
86,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.046389,0.011861,0.005167,0.111392,9.391152,0.004617,1.112008
30,(PAO SOURDOUGH OVOS),(BOWL DE IOGURTE FRUTAS AVEIA),0.027246,0.031709,0.007868,0.288793,9.107679,0.007005,1.361476
82,(SUCO DE LARANJA 400ML),(CROISSANT BRIE PARMA MEL DE),0.024427,0.045449,0.005989,0.245192,5.394864,0.004879,1.264628
96,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.046741,0.022901,0.005402,0.115578,5.046901,0.004332,1.104788
116,"(ADICIONAL DE PRESUNTO, ADICIONAL DE MUSSARELA)",(CROISSANT),0.006107,0.206342,0.005637,0.923077,4.473534,0.004377,10.317557
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.206342,0.007634,0.006929,0.033580,4.398976,0.005354,1.026848
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.206342,0.011274,0.009395,0.045532,4.038607,0.007069,1.035892


In [149]:
morning_method_2 = apriori_results_method_2(hora_do_dia = 'manhã')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [150]:
morning_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
216,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.013316,0.016423,0.009321,0.700000,42.624324,0.009102,3.278592
217,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.015091,0.014647,0.009321,0.617647,42.168449,0.009100,2.577077
0,(ADICIONAL DE PRESUNTO),(ADICIONAL DE MUSSARELA),0.014647,0.016423,0.009765,0.666667,40.594595,0.009524,2.950732
132,(CROISSANT IFOOD),(COOKIES 80G IFOOD),0.081225,0.009321,0.005770,0.071038,7.621390,0.005013,1.066437
176,(CROISSANT IFOOD),(SOURDOUGH 369G IFOOD),0.081225,0.014203,0.008433,0.103825,7.309939,0.007280,1.100005
174,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.081225,0.015535,0.008433,0.103825,6.683372,0.007171,1.098519
178,(SUCO DE LARANJA 400ML),(CROISSANT MUSSATRELA DE),0.036840,0.027075,0.006658,0.180723,6.674896,0.005660,1.187541
170,(SUCO DE LARANJA 400ML),(CROISSANT BRIE PARMA MEL DE),0.036840,0.039947,0.008877,0.240964,6.032129,0.007405,1.264832
196,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.050599,0.024856,0.007102,0.140351,5.646617,0.005844,1.134351
248,"(CROISSANT, PAO SOURDOUGH OVOS)",(CAFE CATUAI VERMELHO G 200ML),0.017310,0.055038,0.005326,0.307692,5.590571,0.004374,1.364946


In [151]:
afternoon_method_2 = apriori_results_method_2(hora_do_dia = 'tarde')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [152]:
afternoon_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
119,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.008975,0.007085,0.006613,0.736842,103.992982,0.006550,3.773075
118,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.006613,0.010392,0.006613,1.000000,96.227273,0.006544,inf
0,(ADICIONAL DE PRESUNTO),(ADICIONAL DE MUSSARELA),0.007085,0.010392,0.007085,1.000000,96.227273,0.007012,inf
84,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.045820,0.011809,0.005196,0.113402,9.602887,0.004655,1.114587
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.168162,0.007085,0.006613,0.039326,5.550187,0.005422,1.033560
32,(CAFE EQUILIBRADO CAPSULA P),(CAFE INTENSO CAPSULA P 50ML),0.042985,0.030704,0.007085,0.164835,5.368555,0.005766,1.160605
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.168162,0.010392,0.008975,0.053371,5.135725,0.007227,1.045402
96,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.053377,0.032121,0.008030,0.150442,4.683628,0.006316,1.139274
12,(BOMBOLONI NUTELLA 80G),(BOMBOLONI DOCE DE LEITE 80G),0.048181,0.059046,0.012754,0.264706,4.483059,0.009909,1.279698
64,(CROISSANT),(LARANJA 400 ML),0.168162,0.028342,0.017005,0.101124,3.567978,0.012239,1.080970


In [153]:
evening_method_2 = apriori_results_method_2(hora_do_dia = 'noite')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [154]:
evening_method_2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
92,(SUCO DE LARANJA 400ML),(CROISSANT BRIE PARMA MEL DE),0.024367,0.055247,0.006273,0.257426,4.659518,0.004926,1.272267
0,(CROISSANT),(ADICIONAL DE MUSSARELA),0.188661,0.008926,0.006514,0.034527,3.867941,0.004830,1.026516
12,(AGUA MINERAL SEM GAS 500 ML),(CROISSANT BRIE PARMA MEL DE),0.040290,0.055247,0.007479,0.185629,3.359961,0.005253,1.160101
14,(CROISSANT MUSSATRELA DE),(AGUA MINERAL SEM GAS 500 ML),0.038601,0.040290,0.005066,0.131250,3.257672,0.003511,1.104703
74,(CROISSANT),(LARANJA 400 ML),0.188661,0.020024,0.011821,0.062660,3.129218,0.008044,1.045486
88,(CROISSANT MUSSATRELA DE),(CROISSANT BRIE PARMA MEL DE),0.038601,0.055247,0.006514,0.168750,3.054449,0.004381,1.136545
4,(AGUA MINERAL S/GAS 500ML),(BOMBOLONI NUTELLA 80G),0.042461,0.046080,0.005549,0.130682,2.836001,0.003592,1.097320
128,(PIZZA AL TAGLIO GORGONZOLA E),(PIZZA AL TAGLIO PEPPERONI 210G),0.055247,0.034499,0.005066,0.091703,2.658106,0.003160,1.062979
18,(BOMBOLONI NUTELLA 80G),(BOMBOLONI DOCE DE LEITE 80G),0.046080,0.056936,0.006755,0.146597,2.574763,0.004132,1.105063
50,(PIZZA AL TAGLIO GORGONZOLA E),(COCA COLA LATA 350 ML),0.055247,0.049940,0.006996,0.126638,2.535810,0.004237,1.087819


In [155]:
weekend_method_2 = apriori_results_method_2(dia_da_semana = 'FDS')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [156]:
weekend_method_2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
175,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.009426,0.006480,0.005007,0.531250,81.981534,0.004946,2.119509
0,(ADICIONAL DE PRESUNTO),(ADICIONAL DE MUSSARELA),0.006480,0.010604,0.005007,0.772727,72.872475,0.004939,4.353343
130,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.064507,0.012077,0.006186,0.095890,7.940194,0.005407,1.092703
50,(PAO SOURDOUGH OVOS),(BOWL DE IOGURTE FRUTAS AVEIA),0.032695,0.043004,0.009426,0.288288,6.703690,0.008020,1.344639
124,(SUCO DE LARANJA 400ML),(CROISSANT BRIE PARMA MEL DE),0.027982,0.047717,0.007658,0.273684,5.735543,0.006323,1.311114
152,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.045066,0.022975,0.005302,0.117647,5.120664,0.004267,1.107295
134,(SUCO DE LARANJA 400ML),(CROISSANT MUSSATRELA DE),0.027982,0.036230,0.005007,0.178947,4.939238,0.003994,1.173823
68,(CAFE EQUILIBRADO CAPSULA P),(CAFE INTENSO CAPSULA P 50ML),0.047717,0.022091,0.005007,0.104938,4.750206,0.003953,1.092560
176,"(ADICIONAL DE PRESUNTO, ADICIONAL DE MUSSARELA)",(CROISSANT),0.005007,0.233284,0.005007,1.000000,4.286616,0.003839,inf
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.233284,0.006480,0.006480,0.027778,4.286616,0.004968,1.021906


In [157]:
weekday_method_2 = apriori_results_method_2(dia_da_semana = 'DDS')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [158]:
weekday_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
103,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.009375,0.008398,0.006055,0.645833,76.899225,0.005976,2.799816
102,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.007227,0.011719,0.006055,0.837838,71.495495,0.005970,6.094401
0,(ADICIONAL DE PRESUNTO),(ADICIONAL DE MUSSARELA),0.008398,0.011719,0.006836,0.813953,69.457364,0.006738,5.312012
26,(PAO SOURDOUGH OVOS),(BOWL DE IOGURTE FRUTAS AVEIA),0.023633,0.024219,0.006836,0.289256,11.943482,0.006264,1.372902
86,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.047852,0.022852,0.005469,0.114286,5.001221,0.004375,1.103232
104,"(ADICIONAL DE PRESUNTO, ADICIONAL DE MUSSARELA)",(CROISSANT),0.006836,0.188477,0.006055,0.885714,4.699334,0.004766,7.100830
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.188477,0.008398,0.007227,0.038342,4.565369,0.005644,1.031137
12,(AGUA MINERAL SEM GAS 500 ML),(CROISSANT BRIE PARMA MEL DE),0.033594,0.043945,0.006641,0.197674,4.498191,0.005164,1.191604
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.188477,0.011719,0.009375,0.049741,4.244560,0.007166,1.040012
16,(BOMBOLONI NUTELLA 80G),(BOMBOLONI DOCE DE LEITE 80G),0.041016,0.046484,0.007227,0.176190,3.790316,0.005320,1.157447


In [159]:
weekend_morning_method_2 = apriori_results_method_2(dia_da_semana = 'FDS', hora_do_dia = 'manhã')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [160]:
weekend_morning_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
267,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.010824,0.009992,0.005828,0.538462,53.891026,0.005720,2.145018
0,(ADICIONAL DE PRESUNTO),(ADICIONAL DE MUSSARELA),0.009992,0.011657,0.005828,0.583333,50.041667,0.005712,2.372023
142,(CIABATTA 123G),(OVOS COM BACON E BRIE),0.019983,0.024147,0.006661,0.333333,13.804598,0.006179,1.463780
220,(SUCO DE LARANJA 400ML),(CROISSANT OVOS),0.044963,0.017485,0.007494,0.166667,9.531746,0.006708,1.179017
285,"(AGUA MINERAL S/GAS 500ML, PASTEL DE NATA 60G)",(SANDUICHE DE PEPERONI BRIE),0.009159,0.067444,0.005828,0.636364,9.435466,0.005211,2.564530
64,(CROISSANT IFOOD),(BOMBOLONI NUTELLA 80G IFOOD),0.098251,0.006661,0.005828,0.059322,8.905720,0.005174,1.055982
128,(CAFE EQUILIBRADO CAPSULA G),(CROISSANT OVOS),0.043297,0.017485,0.005828,0.134615,7.698718,0.005071,1.135350
40,(CROISSANT OVOS),(AGUA MINERAL SEM GAS 500 ML),0.017485,0.044963,0.005828,0.333333,7.413580,0.005042,1.432556
214,(CROISSANT IFOOD),(SOURDOUGH 369G IFOOD),0.098251,0.015820,0.010824,0.110169,6.963872,0.009270,1.106031
238,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.062448,0.024979,0.009992,0.160000,6.405333,0.008432,1.160739


In [161]:
weekday_afternoon_method_2 = apriori_results_method_2(dia_da_semana = 'DDS', hora_do_dia = 'tarde')

c:\Users\ctobr\miniconda3\envs\minimal_ds\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [162]:
weekday_afternoon_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
147,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.006613,0.005878,0.005143,0.777778,132.319444,0.005104,4.473549
0,(ADICIONAL DE PRESUNTO),(ADICIONAL DE MUSSARELA),0.005878,0.008082,0.005878,1.000000,123.727273,0.005831,inf
146,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.005143,0.008082,0.005143,1.000000,123.727273,0.005102,inf
114,(CROISSANT IFOOD),(PASTEL DE NATA 60G IFOOD),0.033064,0.014695,0.006613,0.200000,13.610000,0.006127,1.231631
154,"(BOMBOLONI NUTELLA 80G, PASTEL DE NATA 60G)",(FOLHADO FRANGO 160G),0.008082,0.053637,0.005143,0.636364,11.864259,0.004710,2.602498
153,"(FOLHADO FRANGO 160G, PASTEL DE NATA 60G)",(BOMBOLONI NUTELLA 80G),0.013226,0.046289,0.005143,0.388889,8.401235,0.004531,1.560617
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.161646,0.005878,0.005143,0.031818,5.413068,0.004193,1.026793
42,(CAFE EQUILIBRADO CAPSULA P),(CAFE INTENSO CAPSULA P 50ML),0.043350,0.032329,0.007348,0.169492,5.242681,0.005946,1.165155
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.161646,0.008082,0.006613,0.040909,5.061570,0.005306,1.034227
16,(BOMBOLONI NUTELLA 80G),(BOMBOLONI DOCE DE LEITE 80G),0.046289,0.050698,0.011756,0.253968,5.009432,0.009409,1.272469


In [163]:
weekday_afternoon_method_2.groupby(by='antecedents')['lift'].sum().sort_values(ascending=False)

antecedents
(CROISSANT, ADICIONAL DE MUSSARELA)             132.319444
(ADICIONAL DE PRESUNTO)                         123.727273
(CROISSANT, ADICIONAL DE PRESUNTO)              123.727273
(CROISSANT)                                      31.679994
(COOKIES 80G)                                    17.040575
(CROISSANT IFOOD)                                16.151550
(BOMBOLONI NUTELLA 80G, PASTEL DE NATA 60G)      11.864259
(FOLHADO FRANGO 160G, PASTEL DE NATA 60G)        10.807043
(CAFE EQUILIBRADO CAPSULA P)                     10.160819
(FOLHADO FRANGO 160G)                            10.016827
(BOMBOLONI NUTELLA 80G)                           9.448654
(FOLHADO MACA 80G)                                7.709377
(MINI CROISSANT 25G KIT C 5)                      7.571816
(CROISSANT  TRADICIONAL 80 G)                     7.438063
(FOLHADO FRANGO 160G, BOMBOLONI NUTELLA 80G)      4.472770
(PASTEL DE NATA 60G)                              3.532840
(SANDUICHE DE PEPERONI BRIE)                